# Ex ? Manipulating Spectra Part 2 - `Spectra`

## Notebook Initialization

In [1]:
%load_ext autoreload
import sys
sys.path.append("..")

In [2]:
%matplotlib inline

In [3]:
%autoreload


import matplotlib
matplotlib.rc_file('matplotlibrc')
import matplotlib.pyplot as plt

import numpy as np
from scipy.integrate import quad

import darkhistory.physics as phys
import darkhistory.spec.spectools as spectools
from darkhistory.spec.spectrum import Spectruma
from darkhistory.spec.spectra import Spectra

In [4]:
eng = np.array([1, 10, 100, 1000])
spec_arr = [Spectrum(eng, np.ones(4)*i, rs=100, spec_type='N') for i in np.arange(4)]
test_spectra = Spectra(spec_arr)
test_spectra.redshift(np.array([0.01, 0.1, 1, 10]))
print(test_spectra.grid_vals)

[[0. 0. 0. 0.]
 [1. 0. 0. 0.]
 [2. 2. 0. 0.]
 [3. 3. 3. 0.]]


In [5]:
a = Spectrum(eng, np.ones(4)*3, rs=100, spec_type='N') 
a.totN(bound_type='bin')

array([3., 3., 3., 3.])

In [6]:
0.5*13*10 + 14*100 + 15*1000

16465.0

In [7]:
9*0.5 + 10 + 11

25.5

In [8]:
12*1 + 13*10 + 14*100 + 15*1000

16542

In [9]:
1*10 + 2*100

210

In [10]:
%autoreload 2
phys.Tm_std(100)

array(0.01421714)

In [11]:
a = phys.struct_boost_func()

In [12]:
a(1)

17090553.829749867

In [16]:
%autoreload
phys.f_std(3e5, 300, inj_particle='phot', inj_type='decay', channel='heat')

0.026887206624840804

In [17]:
from config import load_data

In [19]:
a = load_data('dep_tf')

****** Loading transfer functions... ******
    for propagating photons...   Done!
    for low-energy photons...  Done!
    for low-energy electrons...  Done!
    for high-energy deposition...  Done!
    for total upscattered CMB energy rate...  Done!
****** Loading complete! ******


In [36]:
hep_tf = a['highengphot']
lep_tf  = a['lowengphot']
lee_tf  = a['lowengelec']

In [51]:
%%cython

def coarsen_matrices(hep, lep, lee, cf):
    prop_tf = np.zeros_like(hep)
    for i in np.arange(cf):
        prop_tf += np.linalg.matrix_power(hep, i)
    lep = np.matmul(prop_tf, lep)
    lee = np.matmul(prop_tf, lee)
    hep = np.linalg.matrix_power(hep, cf)




Error compiling Cython file:
------------------------------------------------------------
...

def coarsen_matrices(hep, lep, lee, cf):
    prop_tf = np.zeros_like(hep)
             ^
------------------------------------------------------------

/Users/hongwan/.ipython/cython/_cython_magic_b361c7d14b275e957a0cb6ae36816e8b.pyx:3:14: undeclared name not builtin: np


TypeError: object of type 'NoneType' has no len()

In [ ]:
from tqdm import tqdm_notebook as tqdm

cf = 12

for (hep_reg, lep_reg, lee_reg) in zip(hep_tf.grid_vals, lep_tf.grid_vals, lee_tf.grid_vals):
    for (hep_dim_1, lep_dim_1, lee_dim_1) in zip(tqdm(hep_reg), lep_reg, lee_reg):
        if hep_dim_1.ndim == 2:
            coarsen_matrices(hep_dim_1, lep_dim_1, lee_dim_1, cf)
        else:
            for (hep_dim_2, lep_dim_2, lee_dim_2) in zip(tqdm(hep_dim_1), lep_dim_1, lee_dim_1):
                if hep_dim_2.ndim == 2:
                    coarsen_matrices(hep_dim_2, lep_dim_2, lee_dim_2, cf)
                else:
                    for (hep_dim_3, lep_dim_3, lee_dim_3) in zip(hep_dim_2, lep_dim_2, lee_dim_2):
                        coarsen_matrices(hep_dim_3, lep_dim_3, lee_dim_3, cf)
        
            